# Cel analizy symulacji kalorymetru LUXE

### Sformułowanie problemu badawczego 

Eksperyment LUXE (Laser Und XFEL Experiment) zajmuje się badaniem elektrodynamiki kwantowej w silnych polach.  Gdy wysokoenergetyczny pozyton trafia w detektor (złożony z naprzemiennych warstw absorbera-wolframu i sensora-krzemu), nie deponuje energii w jednym punkcie, lecz inicjuje kaskadę elektromagnetyczną. Energia cząstki pierwotnej zostaje rozproszona na wiele komórek (pikseli) w kolejnych warstwach detektora, tworząc chmurę depozytów energii.

Głównym wyzwaniem badawczym jest precyzyjna rekonstrukcja parametrów cząstki pierwotnej na podstawie rozmytej struktury depozytów energii.

### Cel projektu
Celem pracy jest opracowanie, zaimplementowanie i przetestowanie algorytmu rekonstrukcji, który na podstawie zarejestrowanego rozkładu energii w kalorymetrze pozwoli wyznaczyć:
- Pozycję uderzenia cząstki $(x,y)$, czyli punkt wejścia w detektor
- Kierunek lotu cząstki $(\theta_x,\theta_y)$, czyli kąt padania na detektor 


# Budowa kalorymetru 
![Schemat budowy kalorymetru LUXE](Schemat_luxe.png)

### Wizualizacja zdarzenia w detektorze 
Analizując schemat przejścia cząstki przez detektor, obserwujemy proces rozwoju kaskady elektromagnetycznej w układzie wielowarstwowym. 
- Kąt padania: cząstka (pozyton) wpada do detektora pod określonym kątem względem osi prostopadłej do powierzchni czołowej. W zależności od zestawu danych, kąt ten może wynosić $0^\circ$ (padanie prostopadłe) lub być odchylony w zakresie $\pm 10^\circ$ / $\pm 20^\circ$.
- Punkt uderzenia: miejsce interakcji cząstki pierwotnej z pierwszą warstwą detektora (współrzędne $x, y$).
- Struktura warstwowa: widzimy, że detektor nie jest jednolitym blokiem, lecz składa się z 25 odseparowanych warstw. Cząstka, penetrując kolejne warstwy, traci energię w absorberze i generuje cząstki wtórne, których ślad jest próbkowany przez warstwy krzemu.

### Szczegółowa budowa warstwy ("kanapka" wolfram-krzem) 

Każda z 25 warstw składa się z trzech kluczowych elementów: 
- Absorber (wolfram): płyta o grubości 3.5 mm (oś $Z$).  To tutaj zachodzą główne interakcje cząstki z materią i rozwój kaskady.
- Szczelina montażowa: cienka warstwa o grubości 1mm, w której umieszczone są sensory krzemowe (warstwa detekcyjna) o grubości 320 $\mu m$ (oś Z)

**Geometria i wymiary**  

Wolfram jest jedną dużą płytą o całkowitym wymiarze 540 mm $\times$ 90 mm. Pojedynczy sensor ma wymiar 90 mm $\times$ 90 mm. Ponieważ warstwa ma szerokość 540 mm, potrzebujemy dokładnie 6 sensorów ułożonych obok siebie, aby pokryć całą szerokość. Tak więc, rozmiar całej jednej warstwy warstwy to 540 mm (oś $X$) $\times$ 90 mm (oś $Y$) 

**Przeliczanie pikseli na milimetry** 

Każdy sensor podzielony jest na matrycę 16 $\times$ 16 pikseli. Stąd, w jednej warstwie znajduje się 96 pikseli w osi $X$ (6 sensorów) oraz 16 pikseli w osi $Y$. 

Aby przełożyć indeks piksela na fizyczną pozycję w milimetrach, musimy znać rozmiar pojedynczego piksela:
$$\text{Szerokość piksela} = \frac{\text{Szerokość całkowita}}{\text{Liczba pikseli w poziomie}} = \frac{540 \text{ mm}}{96} = 5,625 \text{ mm}$$
$$\text{Wysokość piksela} = \frac{\text{Wysokość całkowita}}{\text{Liczba pikseli w pionie}} = \frac{90 \text{ mm}}{16} = 5,625 \text{ mm}$$
Zatem każdy piksel (cela) to kwadrat o boku 5.625 mm.




### Wyidealizowany model kalorymetru 
Symulacja Monte Carlo, na której pracujemy, odwzorowuje wyidealizowany kalorymetr. Oznacza to, że wirtualny model jest pozbawiony niedoskonałości technicznych, które występowały w rzeczywistym prototypie. 
- Pełna sprawność kanałów: w symulacji zakładamy, że wszystkie kanały działają, co oznacza brak martwych pikseli i uszkodzonej elektroniki – każdy fragment krzemu poprawnie rejestruje energię.
- Głębokość detektora: model symulacyjny jest "głębszy" – posiada 25 warstw, co pozwala na pełniejsze zarejestrowanie profilu kaskady. Stąd też, pełne wymiary wyidealizowanego kalorymetru to 540 mm $\times$ 90 mm $\times$ 112.5 mm 

# Struktura plików z danymi 
**Parametry wiązki**: 

Symulacja zakłada, że wiązka pozytonów jest celowana w geometryczny środek kalorymetru, jednak dla oddania realizmu wprowadzono losowe rozmycia: 
- Rozmycie przestrzenne: Pozycja startowa $(X,Y)$ jest losowana z rozkładu płaskiego. Poziomo $\pm$ 15 mm od środka, pionowo $\pm$ 7,5 mm od środka. 
- Rozmycie kątowe: Kierunek wejścia cząstki również podlega fluktuacjom, co definiuje podział na trzy katalogi danych:
    - `output_0`: Cząstki wpadają niemal prostopadle (brak rozmycia kątowego).
    - `output_10`: Kąt wpadania losowany w zakresie $\pm 10^\circ$. 
    - `output_20`: Kąt wpadania losowany w zakresie $\pm 20^\circ$.


Wartości energii nie są podawane w GeV, lecz w jednostkach MIP (Minimum Ionizing Particle). 1 MIP to najbardziej prawdopodobna wartość sygnału wygenerowanego przez pojedynczą cząstkę minimalnie jonizującą (np. mion) przelatującą prostopadle przez sensor krzemowy o danej grubości. Dzięki tej jednostce dane są znormalizowane i łatwiejsze do interpretacji w kontekście liczby cząstek przechodzących przez dany piksel (np. sygnał 10 MIP oznacza, że przez piksel przeleciało w przybliżeniu 10 cząstek kaskady).

# Etapy realizacji zadania 

**Etap I: Rekonstrukcja samej pozycji**  

Celem tego etapu było znalezienie optymalnych parametrów pracy algorytmu w warunkach idealnych (prostopadłe padanie cząstki). Przeprowadzono optymalizację parametru $E_{cut}$, czyli progu odcięcia energii, w celu wyeliminowania wpływu szumu elektroniki i ogonów kaskady na precyzję wyznaczania poprzecznego położenia (X, Y) rdzenia uderzenia.

**ETAP II: Rekonstrukcja kierunku (analiza kątowa)**  

W tym etapie rozszerzono analizę o trzeci wymiar (głębokość detektora Z), aby odtworzyć tor lotu cząstki, wykorzystując fakt budowy segmentowej kalorymetru. 
 - Wykorzystano zbiory danych z rozmyciem kątowym `output_10` ($\pm 10^\circ$) oraz `output_20` ($\pm 20^\circ$).
 - Zrezygnowano z wysokich wartości $E_{cut}$ wyznaczonych w Etapie I na rzecz znacznie obniżonego progu odcięcia ($0.5$ MIP). Pozwoliło to na wydłużenie obserwowalnego śladu kaskady. Aby zrekompensować wpuszczenie szumu z obrzeży, zastosowano algorytm klasteryzacji, zawężający obliczenia środka ciężkości tylko do bezpośredniego sąsiedztwa najsilniejszego sygnału w każdej warstwie.
 - Wyznaczono trajektorię cząstki poprzez ważone dopasowanie linii prostej do wyznaczonych punktów, przypisując większe znaczenie warstwom o silniejszym, wyraźniejszym depozycie energii.
 - Zbadano korelację między głębokością penetracji kaskady (liczbą aktywnych warstw) a precyzją rekonstrukcji kąta. Zweryfikowano hipotezę, że dłuższy ślad w detektorze przekłada się na znacznie mniejszą niepewność pomiarową, a sam algorytm pozostaje stabilny nawet przy dużych wychyleniach wiązki.